In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import matplotlib as mpl
import seaborn as sb
import numpy as np

In [2]:
def plot(get_chart, xlabel=None, ylabel=None, title=None , figsize=(10,6)):
    titledict = {'fontweight':'bold', 'fontsize': 15}
    labeldict = {'fontsize': 13}
    plt.figure(figsize = figsize)
    ax = get_chart()
    title and ax.set_title(title + "\n", fontdict=titledict)
    xlabel and ax.set_xlabel(xlabel, fontdict=labeldict)
    xlabel and ax.set_ylabel(ylabel, fontdict=labeldict)
    plt.show()

In [3]:
# lista de todos os arquivos
caged_files = sorted(glob('CAGED_DATA\CAGEDMOV*.txt'))
# print(caged_files)
pandemia_df = pd.concat((pd.read_csv(file, sep=';') for file in caged_files), ignore_index=True)

In [4]:
#Lista todos os arquivos carregados e concatenados no dataframe pandemia_df
caged_files

['CAGED_DATA\\CAGEDMOV202003.txt',
 'CAGED_DATA\\CAGEDMOV202004.txt',
 'CAGED_DATA\\CAGEDMOV202005.txt',
 'CAGED_DATA\\CAGEDMOV202006.txt',
 'CAGED_DATA\\CAGEDMOV202007.txt',
 'CAGED_DATA\\CAGEDMOV202008.txt',
 'CAGED_DATA\\CAGEDMOV202009.txt',
 'CAGED_DATA\\CAGEDMOV202010.txt',
 'CAGED_DATA\\CAGEDMOV202011.txt',
 'CAGED_DATA\\CAGEDMOV202012.txt',
 'CAGED_DATA\\CAGEDMOV202101.txt',
 'CAGED_DATA\\CAGEDMOV202102.txt',
 'CAGED_DATA\\CAGEDMOV202103.txt']

In [5]:
#Lista as colunas do dataframe
pandemia_df.columns

Index(['competência', 'região', 'uf', 'município', 'seção', 'subclasse',
       'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução',
       'idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência',
       'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan',
       'indicadoraprendiz', 'fonte'],
      dtype='object')

In [6]:
#Quero poder visualizar as causas de demissão formalizadas pelos empregadores distribuídas por sexo, raça/cor e faixa etária

In [7]:
pandemia_df.tail()

,competência,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,fonte
33797762,202103,3,35,355030,A,151201,1,724315,102,7,...,2,3,10,0,0,0,1500.000000,99,0,1
33797763,202103,3,35,355030,I,5620101,1,513205,111,7,...,0,1,20,0,1,0,6567.000000,7,0,1
33797764,202103,2,25,251370,C,1532700,-1,764105,101,7,...,0,1,31,0,0,0,1122.000000,10,0,1
33797765,202103,3,33,330455,I,5510801,-1,513505,101,6,...,0,1,31,0,0,0,1021.419983,5,0,1
33797766,202103,3,35,354980,G,4530701,-1,142105,101,9,...,0,1,31,0,0,0,4500.000000,4,0,1


In [8]:
pandemia_df['descsexo']= pandemia_df['sexo'].map({
1:'Masculino',
3:'Feminino',
9:'Nao Identificado'})
pandemia_df['descraca'] = pandemia_df['raçacor'].map({
1:'Branca',
2:'Preta',
3:'Parda',
4:'Amarela',
5:'Indigena',
6:'Nao Informada',
9:'Nao Identificado'})
pandemia_df['desctipomovimentação'] = pandemia_df['tipomovimentação'].map({
10:'Admissão por Primeiro Emprego',
20:'Admissão por Reemprego', 
25:'Admissão por contrato trabalho prazo determinado',
31:'Desligamento por Demissão sem Justa Causa', 
32:'Desligamento por Demissão com Justa Causa', 
33:'Culpa Recíproca',
35:'Admissão por Reintegraçao', 
40:'Desligamento a Pedido',
43:'Término contrato trabalho prazo determinado',
45:'Desligamento por Término de Contrato',
50:'Desligamento por Aposentadoria', 
60:'Desligamento por morte',
70:'Admissão por Transferência', 
80:'Desligamento por Transferência', 
90:'Desliamento por Acordo Empregado e Empregador',
98:'Desligamento de Tipo Ignorado',
99:'Não Identificado'})

In [17]:
conditions = [
    (pandemia_df['tipomovimentação'].isin([10,20,25,35,70])),
    (pandemia_df['tipomovimentação'].isin([31,32,33,40,43,45,50,60,80,90,98])),
    (pandemia_df['tipomovimentação'].isin([99]))
    ]

values = ['Admissão', 'Desligamento', 'Não identificado']

pandemia_df['Dados_de_movimentação'] = np.select(conditions, values)

MemoryError: Unable to allocate 2.01 GiB for an array with shape (33797767,) and data type <U16

In [16]:
plot(
    get_chart = lambda: sb.countplot(x='descsexo', hue='Dados_de_movimentação',data = pandemia_df, palette='Paired'),
    title = "Numero de colaboradores admitidos por sexo",
    xlabel = "Sexo",
    ylabel = "Contagem",
)

ValueError: Could not interpret input 'Dados_de_movimentação'

<Figure size 720x432 with 0 Axes>

In [11]:
conditions = [
    ((pandemia_df['idade'] >= 0) & (pandemia_df['idade'] <= 19)),
    ((pandemia_df['idade'] >= 20) & (pandemia_df['idade'] <= 59)),
    (pandemia_df['idade'] >= 60)
    ]

values = ['Jovem', 'Adulto', 'Idoso']

pandemia_df['faixa_etaria'] = np.select(conditions, values)

In [12]:
pandemia_df[pandemia_df['faixa_etaria']=='0']['idade']

MemoryError: Unable to allocate 5.29 GiB for an array with shape (21, 33797767) and data type int64

In [ ]:
def get_chart():
    ax = sb.countplot(x='faixa_etaria',hue='movimento',data = pandemia_df, palette='husl')
    ax.set(yscale="log")
    return ax
plot(
    get_chart = get_chart,
    xlabel = "Faixa Etária",
    ylabel = "Contagem(log)",
    title = "Contagem de colaboradores admitidos e contratados por faixa etária" 
)